# Mission to Mars
Web application that scrapes various websites for data related to the Mission to Mars

In [1]:
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re

In [2]:
#Scrape the Nasa News
url_nasa_news = "https://mars.nasa.gov/news/"
nasa_news_response = requests.get(url_nasa_news)
soup = BeautifulSoup(nasa_news_response.text, 'lxml')


In [3]:
#Save News title and paragraph

news_title = soup.find_all("div", class_= "content_title")[0].text
news_p = soup.find_all("div", class_= "rollover_description_inner")[0].text

print(news_title)
print(news_p)



Alabama High School Student Names NASA's Mars Helicopter



Vaneeza Rupani's essay was chosen as the name for the small spacecraft, which will mark NASA's first attempt at powered flight on another planet.



In [4]:
#execute browser for navigation
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#JPL Mars Space Images - Featured Image
url_mars_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_mars_image)
browser.click_link_by_partial_text('FULL IMAGE')
browser.click_link_by_partial_text('more info')
html=browser.html
soup = BeautifulSoup(html, 'html.parser')
image_page = soup.find("figure", class_="lede")
image_url = image_page.find("a")["href"]
featured_image_url =f"https://www.jpl.nasa.gov{image_url}"
print(featured_image_url)

C:\Users\ritik\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA01384_hires.jpg


In [6]:
# Mars Weather
url_mars_weather = "https://twitter.com/marswxreport?lang=en"
response = requests.get(url_mars_weather)
soup = BeautifulSoup(response.text, 'html.parser')
#print(soup)
pattern = re.compile(r'InSight sol')
mars_weather = soup.find('p', text=pattern).text
print(mars_weather)


InSight sol 505 (2020-04-28) low -93.8ºC (-136.8ºF) high -6.6ºC (20.1ºF)
winds from the WNW at 4.2 m/s (9.4 mph) gusting to 13.2 m/s (29.6 mph)
pressure at 6.70 hPa


In [7]:
# Mars Facts
url_mars_facts = "https://space-facts.com/mars/"
mars_facts = pd.read_html(url_mars_facts)
mars_facts_df = pd.DataFrame(mars_facts[0])
mars_facts_df = mars_facts_df.rename(columns={0:"Description", 1:"Value"})
mars_facts_df.set_index('Description', inplace=True)

#store html table
html_table = mars_facts_df.to_html()
html_table.replace("\n", "")
print(html_table)

#save df as html file
mars_facts_df.to_html("mars_facts.html")

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Value</th>
    </tr>
    <tr>
      <th>Description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


In [8]:
# Mars Hemispheres
mars_hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
hemispheres_response = requests.get(mars_hemispheres_url)
soup = BeautifulSoup(hemispheres_response.text, 'html.parser')

#image data
image_data = soup.find_all("div", class_="item")

#hemisphere dictionary
hemisphere_image_urls = []
for image in image_data:
    image_title = image.find("h3").text
    image_url = image.find("a", class_="itemLink product-item")['href']
    full_image_url = f'https://astrogeology.usgs.gov{image_url}'
    
    # find image src
    image_response = requests.get(full_image_url)
    image_soup = BeautifulSoup(image_response.text, 'html.parser')
    image_src = image_soup.find('img', class_="wide-image")["src"]
    img_src_url = f'https://astrogeology.usgs.gov{image_src}'
    
    #add image title and source url to hemisphere dictionary
    hemisphere_image_urls.append({"title":image_title, "img_url": img_src_url })
  
print(hemisphere_image_urls)


[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
